In [2]:
from collections import defaultdict
import heapq

class Graph:
    def __init__(self):
        self.graph = defaultdict(dict)
        self.flows = defaultdict(lambda: defaultdict(int))
        self.capacities = defaultdict(lambda: defaultdict(int))
        
    def add_edge(self, u, v, capacity, cost):
        self.graph[u][v] = cost
        self.graph[v][u] = cost
        self.capacities[u][v] = capacity
        
    def get_residual_graph(self):
        residual = Graph()
        for u in self.graph:
            for v in self.graph[u]:
                # Aresta direta
                if self.flows[u][v] < self.capacities[u][v]:
                    residual.graph[u][v] = self.graph[u][v]
                # Aresta reversa
                if self.flows[u][v] > 0:
                    residual.graph[v][u] = -self.graph[u][v]
        return residual
        
    def dijkstra(self, start, end):
        distances = {node: float('infinity') for node in self.graph}
        distances[start] = 0
        pq = [(0, start)]
        predecessors = {node: None for node in self.graph}
        
        while pq:
            current_distance, current = heapq.heappop(pq)
            
            if current == end:
                break
                
            if current_distance > distances[current]:
                continue
                
            for neighbor in self.graph[current]:
                distance = current_distance + self.graph[current][neighbor]
                
                if distance < distances[neighbor]:
                    distances[neighbor] = distance
                    predecessors[neighbor] = current
                    heapq.heappush(pq, (distance, neighbor))
                    
        if distances[end] == float('infinity'):
            return None, None
            
        # Reconstruir o caminho
        path = []
        current = end
        while current is not None:
            path.append(current)
            current = predecessors[current]
        path.reverse()
        
        return path, distances[end]

def solve_polish_army_problem():
    # Criar grafo
    g = Graph()
    
    # Adicionar arestas do grafo original com suas capacidades
    edges = [
        ('α1', '1', 2, 1), ('α1', '2', 2, 3),
        ('α2', '1', 2, 2), ('α2', '2', 2, 3),
        ('α3', '2', 1, 1),
        ('1', '2', 4, 1),
        ('1', 'β1', 2, 2), ('1', 'β2', 1, 4),
        ('2', 'β1', 2, 3), ('2', 'β2', 1, 5), ('2', 'β3', 1, 2)
    ]
    
    for u, v, cap, cost in edges:
        g.add_edge(u, v, cap, cost)
    
    # Demandas dos destinos
    demands = {
        'β1': 2,  # Precisa de 2 divisões
        'β2': 1,  # Precisa de 1 divisão
        'β3': 1   # Precisa de 1 divisão
    }
    
    # Algoritmo de Sucessivos Caminhos de Custo Mínimo
    total_flow = 0
    total_cost = 0
    paths_used = []
    
    while total_flow < sum(demands.values()):
        # Obter grafo residual
        residual = g.get_residual_graph()
        
        # Encontrar caminho de menor custo para uma demanda não atendida
        min_path = None
        min_cost = float('infinity')
        source_used = None
        target_used = None
        
        for source in ['α1', 'α2', 'α3']:
            for target, demand in demands.items():
                if g.flows[source][target] < demand:
                    path, cost = residual.dijkstra(source, target)
                    if path and cost < min_cost:
                        min_path = path
                        min_cost = cost
                        source_used = source
                        target_used = target
        
        if min_path is None:
            break
            
        # Aumentar fluxo ao longo do caminho
        paths_used.append((source_used, min_path, target_used))
        for i in range(len(min_path) - 1):
            u, v = min_path[i], min_path[i + 1]
            g.flows[u][v] += 1
            g.flows[v][u] -= 1
            
        total_flow += 1
        total_cost += min_cost
    
    return paths_used, total_cost

# Executar a solução
paths, cost = solve_polish_army_problem()

print("\nMovimentações das tropas:")
for source, path, target in paths:
    print(f"Mover divisão de {source} para {target} através do caminho: {' -> '.join(path)}")
print(f"\nCusto total (risco total): {cost}")

KeyError: 'β1'